# ⚙️ SAM 2 Industrial Object Tracking & YOLO Annotation

Notebook này sử dụng **SAM 2 (Segment Anything Model 2)** để:
1. Segment và tracking **các đối tượng/sản phẩm** trên băng truyền từ video
2. Tạo annotation dataset theo format YOLO để training model detection/segmentation cho đối tượng này

## Workflow:
1. **Setup** - Cài đặt SAM 2 và các dependencies
2. **Video Processing** - Extract video thành frames
3. **Interactive Annotation** - Click chọn các đối tượng/sản phẩm trên 1 frame
4. **Propagation** - SAM 2 tự động track qua toàn bộ video
5. **Export** - Xuất dataset theo format YOLO

## 1. Setup môi trường

Cài đặt SAM 2 và các thư viện cần thiết. Cell này sẽ tự động detect môi trường Colab.

In [ ]:
# Detect môi trường Colab
import os
import sys

try:
    import google.colab
    IN_COLAB = True
    print("✅ Đang chạy trên Google Colab")
except ImportError:
    IN_COLAB = False
    print("ℹ️ Đang chạy local (không phải Colab)")

# Cài đặt SAM 2 và dependencies
if IN_COLAB:
    # Cài đặt SAM 2 từ source
    !pip install -q 'git+https://github.com/facebookresearch/sam2.git'
    !pip install -q opencv-python matplotlib ipympl

    # Tải checkpoint model
    !mkdir -p checkpoints
    !wget -q -P checkpoints https://dl.fbaipublicfiles.com/segment_anything_2/092824/sam2.1_hiera_large.pt
    print("✅ Đã cài đặt SAM 2 và tải checkpoint")
else:
    print("ℹ️ Local mode - đảm bảo đã cài đặt SAM 2 và có checkpoint")

In [ ]:
# Import các thư viện cần thiết
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
from pathlib import Path

# Kiểm tra GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("⚠️ GPU không khả dụng, sử dụng CPU (sẽ chậm hơn)")

# Cấu hình dtype cho memory efficiency
if device.type == "cuda":
    # Sử dụng bfloat16 nếu GPU hỗ trợ
    if torch.cuda.get_device_capability()[0] >= 8:
        torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
        if torch.cuda.get_device_properties(0).major >= 8:
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            print("✅ Đã bật TF32 và bfloat16 để tăng hiệu suất")

In [ ]:
# Import SAM 2
from sam2.build_sam import build_sam2_video_predictor

# Cấu hình model
sam2_checkpoint = "checkpoints/sam2.1_hiera_large.pt"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"

# Khởi tạo predictor
predictor = build_sam2_video_predictor(model_cfg, sam2_checkpoint, device=device)
print("✅ Đã load SAM 2 model")

## 2. Video Processing

Upload video và extract thành các frame JPEG. SAM 2 yêu cầu frames được lưu dưới dạng file ảnh.

In [ ]:
# Upload video (Colab) hoặc chỉ định đường dẫn (local)
if IN_COLAB:
    from google.colab import files
    print("📤 Vui lòng upload video của bạn:")
    uploaded = files.upload()
    video_path = list(uploaded.keys())[0]
    print(f"✅ Đã upload: {video_path}")
else:
    # Đường dẫn video local - THAY ĐỔI ĐƯỜNG DẪN NÀY
    video_path = "industrial_video.mp4"  # <-- Thay đổi đường dẫn video của bạn
    print(f"📹 Sử dụng video: {video_path}")

In [ ]:
# Extract frames từ video
frames_dir = "frames_for_object"
os.makedirs(frames_dir, exist_ok=True)

# Cấu hình extract
FRAME_RATE = 5  # Số frame mỗi giây (giảm nếu video dài để tiết kiệm memory)

# Sử dụng ffmpeg để extract frames
!ffmpeg -i "{video_path}" -vf "fps={FRAME_RATE}" -q:v 2 -start_number 0 "{frames_dir}/%05d.jpg" -y -loglevel quiet

# Đếm số frames đã extract
frame_files = sorted([f for f in os.listdir(frames_dir) if f.endswith('.jpg')])
num_frames = len(frame_files)
print(f"✅ Đã extract {num_frames} frames vào thư mục '{frames_dir}/'")

# Hiển thị thông tin video
cap = cv2.VideoCapture(video_path)
original_fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = total_frames / original_fps
cap.release()
print(f"📹 Video gốc: {original_fps:.1f} FPS, {total_frames} frames, {duration:.1f}s")

In [ ]:
# Preview frame đầu tiên
first_frame_path = os.path.join(frames_dir, frame_files[0])
first_frame = Image.open(first_frame_path)
frame_width, frame_height = first_frame.size

plt.figure(figsize=(12, 8))
plt.imshow(first_frame)
plt.title(f"Frame đầu tiên - Size: {frame_width}x{frame_height}")
plt.axis('off')
plt.show()

print(f"📐 Kích thước frame: {frame_width} x {frame_height} pixels")

## 3. Annotation - Nhập Tọa Độ Đối Tượng Trên Nhiều Frame

Hỗ trợ annotation trên **nhiều frame** khác nhau để theo dõi các object mới xuất hiện.

**Cách sử dụng:**
- Xem ảnh frame với grid tọa độ chi tiết (trục X màu đỏ, trục Y màu xanh)
- Xác định vị trí **tâm** của mỗi đối tượng
- Khai báo annotation theo format: `{frame_idx: [(x1, y1), (x2, y2), ...]}`
- Mỗi frame có thể có nhiều object, và bạn có thể thêm object mới ở các frame sau

In [ ]:
# ============================================================
# CẤU HÌNH CÁC FRAME CẦN ANNOTATION
# ============================================================
# Danh sách các frame index cần hiển thị để annotation
# Thêm các frame mà có object MỚI xuất hiện

ANNOTATION_FRAMES = list(range(0, num_frames, 10))  # Thay đổi theo nhu cầu của bạn

# ============================================================

# Hiển thị từng frame với grid tọa độ
for frame_idx in ANNOTATION_FRAMES:
    if frame_idx >= len(frame_files):
        print(f"⚠️ Frame {frame_idx} không tồn tại (chỉ có {len(frame_files)} frames)")
        continue

    annotation_frame_path = os.path.join(frames_dir, frame_files[frame_idx])
    annotation_image = np.array(Image.open(annotation_frame_path))

    h, w = annotation_image.shape[:2]

    # Vẽ ảnh với grid tọa độ chi tiết
    fig, ax = plt.subplots(figsize=(16, 12))
    ax.imshow(annotation_image)

    # Grid lines - mỗi 50 pixels
    grid_step = 50
    for x in range(0, w + 1, grid_step):
        ax.axvline(x, color='cyan', alpha=0.4, linewidth=0.5)
    for y in range(0, h + 1, grid_step):
        ax.axhline(y, color='cyan', alpha=0.4, linewidth=0.5)

    # Labels tọa độ X (trên cùng và dưới cùng) - mỗi 100 pixels
    for x in range(0, w + 1, 100):
        ax.text(x, -15, str(x), color='red', fontsize=9, ha='center', fontweight='bold')
        ax.text(x, h + 15, str(x), color='red', fontsize=9, ha='center', fontweight='bold')

    # Labels tọa độ Y (bên trái và bên phải) - mỗi 100 pixels
    for y in range(0, h + 1, 100):
        ax.text(-25, y, str(y), color='blue', fontsize=9, va='center', fontweight='bold')
        ax.text(w + 25, y, str(y), color='blue', fontsize=9, va='center', fontweight='bold')

    # Thêm cross-hair tại tâm để tham chiếu
    center_x, center_y = w // 2, h // 2
    ax.axvline(center_x, color='yellow', alpha=0.5, linewidth=1, linestyle='--')
    ax.axhline(center_y, color='yellow', alpha=0.5, linewidth=1, linestyle='--')
    ax.plot(center_x, center_y, 'y+', markersize=20, markeredgewidth=2)

    ax.set_xlim(-50, w + 50)
    ax.set_ylim(h + 50, -50)  # Inverted Y axis
    ax.set_xlabel("X coordinate (pixels)", fontsize=12, color='red', fontweight='bold')
    ax.set_ylabel("Y coordinate (pixels)", fontsize=12, color='blue', fontweight='bold')
    ax.set_title(f"🎯 FRAME {frame_idx} - Xác định tọa độ X,Y của đối tượng\n"
                 f"Kích thước: {w}x{h} pixels | Grid: 50px", fontsize=14)

    plt.tight_layout()
    plt.show()

    print(f"📷 Frame {frame_idx}: {frame_files[frame_idx]}")
    print("-" * 50)

print("\n" + "="*70)
print("📝 HƯỚNG DẪN KHAI BÁO TỌA ĐỘ (MULTI-FRAME):")
print("   Xem cell tiếp theo để khai báo tọa độ cho từng frame")
print("="*70)

In [ ]:
# ============================================================
# NHẬP TỌA ĐỘ ĐỐI TƯỢNG TRÊN NHIỀU FRAME
# ============================================================
# Format: { frame_idx: [(x1, y1), (x2, y2), ...] }
#
# - frame_idx: Số thứ tự frame (bắt đầu từ 0)
# - (x, y): Tọa độ tâm của đối tượng
#
# 💡 LƯU Ý QUAN TRỌNG:
# - Frame 0: Đánh dấu TẤT CẢ objects hiện có trong frame đầu tiên
# - Frame 10, 20, ...: Chỉ cần đánh dấu các objects MỚI xuất hiện
# - SAM 2 sẽ tự động track objects đã được đánh dấu ở các frame trước
# - Object ID sẽ được gán tự động theo thứ tự khai báo (1, 2, 3, ...)

multi_frame_annotations = {
    # ============ FRAME 0 - Objects ban đầu ============
    0: [
        (950,600),
        (550,450),
        (350,350),
        (250,250)
    ],

        # ============ FRAME 10 - Objects mới xuất hiện ============
    # 10: [
    #     # (x, y),   # Object mới 1
    #     # (x, y),   # Object mới 2
    # ],

    # ============ FRAME 20 - Objects mới xuất hiện ============
    # 20: [
    #     # (x, y),   # Object mới
    # ],

    # ============ Thêm các frame khác nếu cần ============
    # 30: [
    #     # (x, y),
    # ],

    20: [
        (250,250)
    ],

    40: [
        (275,250)
    ],

    50: [
        (250,200)
    ],

    70: [
        (250,250)
    ],

    80: [
        (225,200)
    ],

    100: [
        (250,200)
    ],

    120: [
        (250,200)
    ],

    130: [
        (200,200)
    ],

    140: [
        (200,200)
    ],

    160: [
        (200,200)
    ],

    190: [
        (250,250)
    ],

    200: [
        (225,200)
    ],

    220: [
        (250,250)
    ],

    240: [
        (250,250)
    ],

    270: [
        (250,250)
    ],

    290: [
        (275,250)
    ],

    300: [
        (250,250)
    ],

    320: [
        (250,250)
    ],


}

# ============================================================
# KIỂM TRA VÀ TỔNG HỢP ANNOTATIONS
# ============================================================

# Đếm tổng số objects và kiểm tra
total_objects = 0
object_counter = 0
all_prompts = {}  # {obj_id: {"frame_idx": idx, "point": (x, y)}}

print("📊 TỔNG HỢP ANNOTATIONS:")
print("=" * 60)

for frame_idx in sorted(multi_frame_annotations.keys()):
    points = multi_frame_annotations[frame_idx]
    if not points:
        continue

    print(f"\n🎯 Frame {frame_idx}:")
    for i, (x, y) in enumerate(points):
        object_counter += 1
        all_prompts[object_counter] = {
            "frame_idx": frame_idx,
            "point": (x, y)
        }
        print(f"   Object {object_counter}: ({x}, {y})")

    total_objects += len(points)

print("\n" + "=" * 60)
if total_objects == 0:
    print("⚠️ CHƯA NHẬP TỌA ĐỘ NÀO!")
    print("   Vui lòng điền tọa độ vào dictionary 'multi_frame_annotations' ở trên")
    print("\n   Ví dụ:")
    print("   multi_frame_annotations = {")
    print("       0: [(150, 200), (350, 180)],   # 2 objects ở frame 0")
    print("       10: [(500, 300)],              # 1 object mới ở frame 10")
    print("   }")
else:
    print(f"✅ Tổng cộng: {total_objects} objects trên {len([k for k, v in multi_frame_annotations.items() if v])} frames")
    print(f"   Object IDs: 1 → {object_counter}")

In [ ]:
# Preview các điểm đã chọn trên từng frame
if total_objects > 0:
    # Nhóm objects theo frame
    frames_to_show = sorted(set(p["frame_idx"] for p in all_prompts.values()))

    for frame_idx in frames_to_show:
        frame_path = os.path.join(frames_dir, frame_files[frame_idx])
        frame_image = np.array(Image.open(frame_path))

        fig, ax = plt.subplots(figsize=(14, 10))
        ax.imshow(frame_image)

        # Vẽ các objects được annotate TẠI frame này
        objects_in_frame = [(obj_id, info) for obj_id, info in all_prompts.items()
                           if info["frame_idx"] == frame_idx]

        for obj_id, info in objects_in_frame:
            x, y = info["point"]
            # Vẽ marker
            ax.plot(x, y, 'r*', markersize=20, markeredgecolor='white', markeredgewidth=1)
            ax.add_patch(plt.Circle((x, y), 25, fill=False, color='lime', linewidth=2))
            # Label
            ax.text(x + 30, y - 10, f"Obj {obj_id}\n({x},{y})", color='yellow', fontsize=10,
                    fontweight='bold', bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))

        ax.set_title(f"✅ Frame {frame_idx}: {len(objects_in_frame)} object(s) được annotate tại đây", fontsize=14)
        ax.axis('off')
        plt.tight_layout()
        plt.show()

    print(f"\n⚙️ Tổng cộng {total_objects} objects sẽ được track bởi SAM 2")
    print("   Tiếp tục chạy các cell tiếp theo để segment và propagate.")
else:
    print("⚠️ Chưa có tọa độ nào. Hãy nhập tọa độ vào cell trước.")

## 4. SAM 2 Segmentation & Tracking

Sử dụng SAM 2 để segment các đối tượng đã click và propagate qua toàn bộ video.

In [ ]:
# Khởi tạo SAM 2 inference state
inference_state = predictor.init_state(video_path=frames_dir)
print(f"✅ Đã khởi tạo inference state với {num_frames} frames")

In [ ]:
# Thêm các prompt clicks cho mỗi đối tượng (HỖ TRỢ MULTI-FRAME)
predictor.reset_state(inference_state)

prompts = {}  # Lưu trữ prompts để visualize

for obj_id, info in all_prompts.items():
    frame_idx = info["frame_idx"]
    x, y = info["point"]

    # Mỗi click là một positive point cho object riêng biệt
    points = np.array([[x, y]], dtype=np.float32)
    labels = np.array([1], dtype=np.int32)  # 1 = positive click

    # Thêm prompt vào SAM 2 TẠI FRAME TƯƠNG ỨNG
    _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
        inference_state=inference_state,
        frame_idx=frame_idx,  # Frame mà object xuất hiện
        obj_id=obj_id,
        points=points,
        labels=labels,
    )

    prompts[obj_id] = {"frame_idx": frame_idx, "point": (x, y)}
    print(f"✅ Object {obj_id} tại frame {frame_idx}: ({x:.1f}, {y:.1f})")

print(f"\n⚙️ Tổng cộng {len(all_prompts)} objects đã được thêm vào SAM 2")
print(f"   Trên {len(set(p['frame_idx'] for p in all_prompts.values()))} frames khác nhau")

In [ ]:
# Hàm hiển thị mask
def show_mask(mask, ax, obj_id=None, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        cmap = plt.cm.get_cmap("tab20")
        color = np.array([*cmap(obj_id % 20)[:3], 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=200):
    pos_points = coords[labels == 1]
    neg_points = coords[labels == 0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*',
               s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*',
               s=marker_size, edgecolor='white', linewidth=1.25)

# Preview segmentation trên CÁC FRAME ĐÃ ANNOTATE
annotated_frames = sorted(set(p["frame_idx"] for p in prompts.values()))

for frame_idx in annotated_frames:
    frame_path = os.path.join(frames_dir, frame_files[frame_idx])
    frame_image = np.array(Image.open(frame_path))

    plt.figure(figsize=(14, 10))
    plt.imshow(frame_image)
    plt.title(f"Preview Segmentation - Frame {frame_idx}")

    # Lấy các objects được annotate tại frame này
    objects_at_frame = {obj_id: info for obj_id, info in prompts.items()
                        if info["frame_idx"] == frame_idx}

    # Vẽ masks và points
    for obj_id, prompt in objects_at_frame.items():
        # Lấy mask từ SAM 2
        _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=frame_idx,
            obj_id=obj_id,
            points=np.array([[prompt["point"][0], prompt["point"][1]]], dtype=np.float32),
            labels=np.array([1], dtype=np.int32),
        )

        # Tìm index của obj_id trong output
        if hasattr(out_obj_ids, 'cpu'):
            obj_ids_np = out_obj_ids.cpu().numpy()
        else:
            obj_ids_np = np.array(out_obj_ids)

        idx = np.where(obj_ids_np == obj_id)[0]
        if len(idx) > 0:
            idx = idx[0]
            mask = (out_mask_logits[idx] > 0.0).cpu().numpy()
            show_mask(mask, plt.gca(), obj_id)

        # Vẽ point
        x, y = prompt["point"]
        plt.plot(x, y, 'g*', markersize=15)
        plt.text(x + 5, y - 5, f"Obj {obj_id}", color='white', fontsize=10, fontweight='bold')

    plt.axis('off')
    plt.tight_layout()
    plt.show()

    print(f"✅ Frame {frame_idx}: {len(objects_at_frame)} objects")

print(f"\n✅ Preview segmentation hoàn tất cho {len(annotated_frames)} frames")

In [ ]:
# Propagate masks qua toàn bộ video - với xử lý lỗi cho multi-frame annotation
# TỐI ƯU: Mỗi object chỉ propagate trong khoảng frame giới hạn

print("🔄 Đang propagate masks qua toàn bộ video...")
print("   (Quá trình này có thể mất vài phút tùy thuộc vào độ dài video)")

video_segments = {}  # {frame_idx: {obj_id: mask}}

# ============================================================
# CẤU HÌNH
# ============================================================
FRAMES_PER_OBJECT = 65  # Số frames tối đa để propagate cho mỗi object

# ============================================================
# PHƯƠNG PHÁP 1: Cố gắng propagate bình thường (cách gốc)
# ============================================================
try:
    print("\n📌 Thử phương pháp 1: Propagate toàn bộ video cùng lúc...")

    for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
        frame_masks = {}
        for i in range(len(out_obj_ids)):
            obj_id = out_obj_ids[i]
            if hasattr(obj_id, 'item'):
                obj_id = obj_id.item()
            else:
                obj_id = int(obj_id)

            mask = (out_mask_logits[i] > 0.0).cpu().numpy().squeeze()
            frame_masks[obj_id] = mask

        video_segments[out_frame_idx] = frame_masks

        if (out_frame_idx + 1) % 10 == 0 or out_frame_idx == num_frames - 1:
            print(f"   Processed frame {out_frame_idx + 1}/{num_frames}")

    print(f"✅ Phương pháp 1 thành công! Có {len(video_segments)} frames với masks")

except (RuntimeError, Exception) as e:
    print(f"\n⚠️ Phương pháp 1 bị lỗi: {str(e)[:100]}...")
    print("🔄 Chuyển sang phương pháp 2: Propagate từng object riêng biệt (TỐI ƯU)...\n")

    # ============================================================
    # PHƯƠNG PHÁP 2: Propagate từng object riêng biệt với giới hạn frames
    # ============================================================
    video_segments = {}  # Reset

    # Sắp xếp objects theo frame_idx để xác định object cuối
    sorted_prompts = sorted(all_prompts.items(), key=lambda x: x[1]["frame_idx"])
    total_objects = len(sorted_prompts)

    for obj_counter, (obj_id, info) in enumerate(sorted_prompts, 1):
        frame_idx = info["frame_idx"]
        x, y = info["point"]

        # Tính toán frame kết thúc
        # Object cuối cùng hoặc các object gần cuối -> propagate đến hết video
        remaining_objects = total_objects - obj_counter
        if remaining_objects <= 2:  # 2 objects cuối cùng propagate đến hết
            end_frame = num_frames
        else:
            end_frame = min(frame_idx + FRAMES_PER_OBJECT, num_frames)

        frames_to_process = end_frame - frame_idx
        print(f"   [{obj_counter}/{total_objects}] Object {obj_id}: frame {frame_idx} → {end_frame-1} ({frames_to_process} frames)")

        try:
            # Reset và khởi tạo lại inference state cho object này
            predictor.reset_state(inference_state)

            # Thêm prompt cho object này
            points = np.array([[x, y]], dtype=np.float32)
            labels = np.array([1], dtype=np.int32)

            predictor.add_new_points_or_box(
                inference_state=inference_state,
                frame_idx=frame_idx,
                obj_id=obj_id,
                points=points,
                labels=labels,
            )

            # Propagate object này - CHỈ LẤY FRAMES TRONG KHOẢNG GIỚI HẠN
            for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
                # Chỉ lưu mask nếu frame nằm trong khoảng cho phép
                if out_frame_idx < frame_idx or out_frame_idx >= end_frame:
                    continue

                if out_frame_idx not in video_segments:
                    video_segments[out_frame_idx] = {}

                for i in range(len(out_obj_ids)):
                    out_obj_id = out_obj_ids[i]
                    if hasattr(out_obj_id, 'item'):
                        out_obj_id = out_obj_id.item()
                    else:
                        out_obj_id = int(out_obj_id)

                    if out_obj_id == obj_id:
                        mask = (out_mask_logits[i] > 0.0).cpu().numpy().squeeze()
                        video_segments[out_frame_idx][obj_id] = mask
                        break

            print(f"      ✅ Object {obj_id} hoàn tất")

        except Exception as e2:
            print(f"      ⚠️ Object {obj_id} bị lỗi: {str(e2)[:80]}")
            continue

    print(f"\n✅ Phương pháp 2 hoàn tất! Có {len(video_segments)} frames với masks")

# Xóa frames rỗng (không có object nào)
frames_to_remove = [f for f in video_segments if len(video_segments[f]) == 0]
for f in frames_to_remove:
    del video_segments[f]

print(f"\n📊 Thống kê cuối cùng:")
print(f"   - Tổng frames có objects: {len(video_segments)}")
print(f"   - Tổng objects: {len(all_prompts)}")
if video_segments:
    total_mask_count = sum(len(masks) for masks in video_segments.values())
    print(f"   - Tổng masks đã tạo: {total_mask_count}")

In [ ]:
# Propagate masks qua toàn bộ video
# TỐI ƯU: Mỗi object chỉ propagate trong khoảng frame giới hạn
# ✅ CHỈ CHẠY PHƯƠNG PHÁP 2: Propagate từng object riêng biệt + DỪNG SỚM

print("🔄 Đang propagate masks qua toàn bộ video...")
print("   (Quá trình này có thể mất vài phút tùy thuộc vào độ dài video)")
print("   📌 Mode: Propagate từng object riêng biệt (Phương pháp 2 + Early Stop)\n")

video_segments = {}  # {frame_idx: {obj_id: mask}}

# ============================================================
# CẤU HÌNH
# ============================================================
FRAMES_PER_OBJECT = 65  # Số frames tối đa để propagate cho mỗi object

# ============================================================
# PHƯƠNG PHÁP 2: Propagate từng object riêng biệt với EARLY STOP
# ============================================================

# Sắp xếp objects theo frame_idx để xác định object cuối
sorted_prompts = sorted(all_prompts.items(), key=lambda x: x[1]["frame_idx"])
total_objects = len(sorted_prompts)

print(f"📊 Tổng số objects: {total_objects}")
print(f"📊 Tổng số frames: {num_frames}")
print(f"📊 Frames per object: {FRAMES_PER_OBJECT}\n")
print("=" * 70)

for obj_counter, (obj_id, info) in enumerate(sorted_prompts, 1):
    frame_idx = info["frame_idx"]
    x, y = info["point"]

    # Tính toán frame kết thúc
    remaining_objects = total_objects - obj_counter
    if remaining_objects <= 2:  # 2 objects cuối cùng propagate đến hết
        end_frame = num_frames
    else:
        end_frame = min(frame_idx + FRAMES_PER_OBJECT, num_frames)

    frames_to_process = end_frame - frame_idx
    print(f"\n[{obj_counter}/{total_objects}] Object {obj_id}")
    print(f"   📍 Xuất hiện tại frame: {frame_idx}")
    print(f"   📍 Tọa độ: ({x}, {y})")
    print(f"   🔄 Propagate từ frame {frame_idx} → {end_frame-1} ({frames_to_process} frames)")

    try:
        # Khởi tạo lại inference state cho object này
        inference_state = predictor.init_state(video_path=frames_dir)

        # Thêm prompt cho object này
        points = np.array([[x, y]], dtype=np.float32)
        labels = np.array([1], dtype=np.int32)

        predictor.add_new_points_or_box(
            inference_state=inference_state,
            frame_idx=frame_idx,
            obj_id=obj_id,
            points=points,
            labels=labels,
        )

        # Propagate object này với EARLY STOP
        masks_found = 0
        for out_frame_idx, out_obj_ids, out_mask_logits in predictor.propagate_in_video(inference_state):
            # ⚡ EARLY STOP: Dừng ngay khi vượt quá end_frame
            if out_frame_idx >= end_frame:
                print(f"   ⚡ Early stop tại frame {out_frame_idx}")
                break

            # Bỏ qua frames trước frame_idx (nếu có)
            if out_frame_idx < frame_idx:
                continue

            if out_frame_idx not in video_segments:
                video_segments[out_frame_idx] = {}

            for i in range(len(out_obj_ids)):
                out_obj_id = out_obj_ids[i]
                if hasattr(out_obj_id, 'item'):
                    out_obj_id = out_obj_id.item()
                else:
                    out_obj_id = int(out_obj_id)

                if out_obj_id == obj_id:
                    mask = (out_mask_logits[i] > 0.0).cpu().numpy().squeeze()
                    video_segments[out_frame_idx][obj_id] = mask
                    masks_found += 1
                    break

        print(f"   ✅ Hoàn tất - Tìm được {masks_found} masks")

    except Exception as e2:
        print(f"   ⚠️ Lỗi: {str(e2)[:100]}")
        continue

print("\n" + "=" * 70)

# Xóa frames rỗng (không có object nào)
frames_to_remove = [f for f in video_segments if len(video_segments[f]) == 0]
for f in frames_to_remove:
    del video_segments[f]

print(f"\n✅ Propagation hoàn tất!\n")
print(f"📊 Thống kê cuối cùng:")
print(f"   - Tổng frames có objects: {len(video_segments)}")
print(f"   - Tổng objects: {len(all_prompts)}")
if video_segments:
    total_mask_count = sum(len(masks) for masks in video_segments.values())
    print(f"   - Tổng masks đã tạo: {total_mask_count}")

    # Chi tiết từng object
    print(f"\n📋 Chi tiết objects:")
    for obj_id in sorted(all_prompts.keys()):
        frames_with_obj = [f for f in video_segments if obj_id in video_segments[f]]
        print(f"   Object {obj_id}: {len(frames_with_obj)} frames")

In [ ]:
# Visualize kết quả trên mỗi 10 frames
print("🎬 Visualize kết quả tracking qua video...\n")

# Tạo danh sách frames để visualize (mỗi 10 frames)
vis_frames = list(range(0, num_frames, 10))
if (num_frames - 1) not in vis_frames:
    vis_frames.append(num_frames - 1)  # Thêm frame cuối

# Tính số hàng cần thiết (mỗi hàng 5 ảnh)
cols = 5
rows = (len(vis_frames) + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(20, 4 * rows))
axes = axes.flatten() if rows > 1 else (axes if cols > 1 else [axes])

for idx, frame_idx in enumerate(vis_frames):
    if idx >= len(axes):
        break

    ax = axes[idx]
    frame_path = os.path.join(frames_dir, frame_files[frame_idx])
    frame = np.array(Image.open(frame_path))
    ax.imshow(frame)

    # Đếm số objects trong frame này
    obj_count = 0
    if frame_idx in video_segments:
        for obj_id, mask in video_segments[frame_idx].items():
            show_mask(mask, ax, obj_id)
            obj_count += 1

    ax.set_title(f"Frame {frame_idx}\n({obj_count} objects)", fontsize=10)
    ax.axis('off')

# Ẩn các axes thừa
for idx in range(len(vis_frames), len(axes)):
    axes[idx].axis('off')
    axes[idx].set_visible(False)

plt.suptitle("⚙️ Kết quả Tracking Đối tượng qua Video (mỗi 10 frames)", fontsize=14)
plt.tight_layout()
plt.show()

# Thống kê chi tiết
print("\n📊 Thống kê chi tiết theo frame:")
print("-" * 50)
for frame_idx in vis_frames:
    if frame_idx in video_segments:
        obj_ids = list(video_segments[frame_idx].keys())
        print(f"   Frame {frame_idx:3d}: {len(obj_ids)} objects - IDs: {obj_ids}")
    else:
        print(f"   Frame {frame_idx:3d}: 0 objects")

## 5. Convert sang YOLO Format

Chuyển đổi segmentation masks sang format YOLO detection/segmentation để training model.

**YOLO Format:** `class_id x_center y_center width height` (normalized 0-1) hoặc `class_id x1 y1 x2 y2 ...`

In [ ]:
def mask_to_yolo_bbox(mask, image_width, image_height, class_id=0):
    """
    Chuyển đổi binary mask sang YOLO bounding box format.

    Args:
        mask: 2D numpy array (H, W) với True/False hoặc 1/0
        image_width: Chiều rộng ảnh
        image_height: Chiều cao ảnh
        class_id: ID của class (mặc định 0 cho 'product')

    Returns:
        String "class_id x_center y_center width height" hoặc None nếu mask rỗng
    """
    # Tìm vị trí các pixel trong mask
    rows = np.any(mask, axis=1)
    cols = np.any(mask, axis=0)

    if not rows.any() or not cols.any():
        return None  # Mask rỗng

    # Tìm bounding box
    y_indices = np.where(rows)[0]
    x_indices = np.where(cols)[0]
    y_min, y_max = y_indices[0], y_indices[-1]
    x_min, x_max = x_indices[0], x_indices[-1]

    # Chuyển sang YOLO format (normalized center + size)
    x_center = ((x_min + x_max) / 2) / image_width
    y_center = ((y_min + y_max) / 2) / image_height
    width = (x_max - x_min) / image_width
    height = (y_max - y_min) / image_height

    return f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}"


def mask_to_yolo_segmentation(mask, image_width, image_height, class_id=0):
    """
    Chuyển đổi binary mask sang YOLO segmentation format (polygon).

    Args:
        mask: 2D numpy array (H, W) với True/False hoặc 1/0
        image_width: Chiều rộng ảnh
        image_height: Chiều cao ảnh
        class_id: ID của class

    Returns:
        String "class_id x1 y1 x2 y2 ... xn yn" (normalized) hoặc None
    """
    # Chuyển mask sang uint8
    mask_uint8 = (mask * 255).astype(np.uint8)

    # Tìm contours
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None

    # Lấy contour lớn nhất
    largest_contour = max(contours, key=cv2.contourArea)

    # Đơn giản hóa contour để giảm số điểm
    epsilon = 0.005 * cv2.arcLength(largest_contour, True)
    approx = cv2.approxPolyDP(largest_contour, epsilon, True)

    if len(approx) < 3:
        return None  # Cần ít nhất 3 điểm cho polygon

    # Chuyển sang normalized coordinates
    points = approx.squeeze()
    normalized_points = []
    for point in points:
        x_norm = point[0] / image_width
        y_norm = point[1] / image_height
        normalized_points.extend([f"{x_norm:.6f}", f"{y_norm:.6f}"])

    return f"{class_id} " + " ".join(normalized_points)


print("✅ Đã định nghĩa các hàm chuyển đổi YOLO format")

In [ ]:
# Cấu hình output
OUTPUT_FORMAT = "detection"  # "detection" (bbox) hoặc "segmentation" (polygon)
CLASS_ID = 0  # Class ID cho đối tượng/sản phẩm
TRAIN_SPLIT = 0.8  # 80% train, 20% val

# Tạo cấu trúc thư mục YOLO
output_dir = "yolo_product_dataset"
os.makedirs(f"{output_dir}/images/train", exist_ok=True)
os.makedirs(f"{output_dir}/images/val", exist_ok=True)
os.makedirs(f"{output_dir}/labels/train", exist_ok=True)
os.makedirs(f"{output_dir}/labels/val", exist_ok=True)

print(f"📁 Đã tạo cấu trúc thư mục YOLO tại '{output_dir}/'")
print(f"   - Format: {OUTPUT_FORMAT}")
print(f"   - Train/Val split: {TRAIN_SPLIT*100:.0f}%/{(1-TRAIN_SPLIT)*100:.0f}%")

In [ ]:
# Export annotations sang YOLO format
# ⏱️ KHÔNG RANDOM - Giữ thứ tự thời gian của video (sequential split)
import shutil

# ============================================================
# CẤU HÌNH TRAIN/VAL SPLIT THEO THỨ TỰ THỜI GIAN
# ============================================================
frame_indices = sorted(list(video_segments.keys()))  # Giữ nguyên thứ tự

# Split train/val theo thứ tự tuần tự (80% trước, 20% sau)
split_idx = int(len(frame_indices) * TRAIN_SPLIT)
train_indices = set(frame_indices[:split_idx])
val_indices = set(frame_indices[split_idx:])

stats = {"train": 0, "val": 0, "total_objects": 0, "empty_frames": 0}

print("📝 Đang export annotations...")
print(f"   📊 Train/Val split (Sequential - theo thời gian):")
print(f"      - Train: frames {frame_indices[0]} → {frame_indices[split_idx-1]} ({len(train_indices)} frames)")
print(f"      - Val: frames {frame_indices[split_idx]} → {frame_indices[-1]} ({len(val_indices)} frames)\n")

for frame_idx in sorted(video_segments.keys()):
    # Xác định train hay val
    split = "train" if frame_idx in train_indices else "val"

    # Đường dẫn files
    frame_name = frame_files[frame_idx]
    label_name = frame_name.replace('.jpg', '.txt')

    src_image = os.path.join(frames_dir, frame_name)
    dst_image = os.path.join(output_dir, "images", split, frame_name)
    dst_label = os.path.join(output_dir, "labels", split, label_name)

    # Copy image
    shutil.copy2(src_image, dst_image)

    # Tạo label file
    labels = []
    for obj_id, mask in video_segments[frame_idx].items():
        if OUTPUT_FORMAT == "detection":
            label = mask_to_yolo_bbox(mask, frame_width, frame_height, CLASS_ID)
        else:
            label = mask_to_yolo_segmentation(mask, frame_width, frame_height, CLASS_ID)

        if label:
            labels.append(label)
            stats["total_objects"] += 1

    # Ghi label file
    if labels:
        with open(dst_label, 'w') as f:
            f.write('\n'.join(labels))
        stats[split] += 1
    else:
        # Tạo file rỗng cho frames không có object
        with open(dst_label, 'w') as f:
            pass
        stats["empty_frames"] += 1

    # Progress
    if (frame_idx + 1) % 20 == 0:
        print(f"   Processed {frame_idx + 1}/{len(video_segments)} frames")

print(f"\n✅ Export hoàn tất!")
print(f"   📊 Thống kê:")
print(f"      - Train images: {stats['train']}")
print(f"      - Val images: {stats['val']}")
print(f"      - Tổng số objects: {stats['total_objects']}")
print(f"      - Empty frames: {stats['empty_frames']}")
print(f"\n   ⏱️ Thứ tự thời gian được giữ nguyên:")
print(f"      Train: những frame sớm trong video")
print(f"      Val: những frame muộn trong video")

In [ ]:
# Tạo file data.yaml cho YOLO training
data_yaml_content = f"""# Industrial Product Detection Dataset
# Generated by SAM 2 Object Tracking Notebook

path: {os.path.abspath(output_dir)}
train: images/train
val: images/val

# Classes
names:
  0: product

# Number of classes
nc: 1
"""

yaml_path = os.path.join(output_dir, "data.yaml")
with open(yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"✅ Đã tạo file data.yaml")
print(f"\n📄 Nội dung data.yaml:")
print("-" * 40)
print(data_yaml_content)

In [ ]:
# Verify một số annotations
print("🔍 Kiểm tra một số annotations mẫu:\n")

# Lấy một vài file label để verify
sample_labels = []
for split in ["train", "val"]:
    label_dir = os.path.join(output_dir, "labels", split)
    labels = [f for f in os.listdir(label_dir) if f.endswith('.txt')]
    if labels:
        sample_labels.append((split, labels[0]))

for split, label_file in sample_labels[:2]:
    label_path = os.path.join(output_dir, "labels", split, label_file)
    image_file = label_file.replace('.txt', '.jpg')
    image_path = os.path.join(output_dir, "images", split, image_file)

    print(f"📁 {split}/{label_file}:")
    with open(label_path, 'r') as f:
        content = f.read().strip()
        if content:
            lines = content.split('\n')
            print(f"   Số objects: {len(lines)}")
            for i, line in enumerate(lines[:3], 1):  # Show max 3 lines
                print(f"   Line {i}: {line[:80]}...")
            if len(lines) > 3:
                print(f"   ... và {len(lines) - 3} objects khác")
        else:
            print("   (Empty - no objects)")
    print()

In [ ]:
# Visualize annotation trên một frame mẫu
def visualize_yolo_annotation(image_path, label_path, title=""):
    """Hiển thị ảnh với bounding boxes từ YOLO annotation"""
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img.shape[:2]

    fig, ax = plt.subplots(figsize=(12, 8))
    ax.imshow(img)

    # Đọc labels
    with open(label_path, 'r') as f:
        lines = f.readlines()

    colors = plt.cm.tab20(np.linspace(0, 1, 20))

    for i, line in enumerate(lines):
        parts = line.strip().split()
        if len(parts) >= 5:  # Detection format
            class_id = int(parts[0])
            x_center, y_center, width, height = map(float, parts[1:5])

            # Convert to pixel coordinates
            x1 = (x_center - width/2) * w
            y1 = (y_center - height/2) * h
            box_w = width * w
            box_h = height * h

            # Draw rectangle
            rect = plt.Rectangle((x1, y1), box_w, box_h,
                                 fill=False, edgecolor=colors[i % 20], linewidth=2)
            ax.add_patch(rect)
            ax.text(x1, y1 - 5, f"product_{i+1}", color=colors[i % 20],
                   fontsize=8, fontweight='bold')

    ax.set_title(f"{title}\n({len(lines)} objects detected)")
    ax.axis('off')
    plt.tight_layout()
    plt.show()

# Visualize sample
for split in ["train", "val"]:
    label_dir = os.path.join(output_dir, "labels", split)
    labels = sorted([f for f in os.listdir(label_dir) if f.endswith('.txt')])

    # Tìm file có annotations
    for label_file in labels[:5]:
        label_path = os.path.join(label_dir, label_file)
        with open(label_path, 'r') as f:
            if f.read().strip():  # Có content
                image_file = label_file.replace('.txt', '.jpg')
                image_path = os.path.join(output_dir, "images", split, image_file)
                visualize_yolo_annotation(image_path, label_path, f"{split.upper()}: {image_file}")
                break

## 6. Download Dataset (Colab)

Tải dataset về máy local hoặc mount Google Drive để lưu trữ.

In [ ]:
import os
from google.colab import files

output_dir = "yolo_product_dataset"
zip_filename = "industrial_product_yolo_dataset.zip"

# Nén dataset thành file zip
!cd {output_dir} && zip -r ../{zip_filename} . -q

print(f"✅ Đã tạo file {zip_filename}")

# Download (chỉ trên Colab)
if IN_COLAB:
    files.download(zip_filename)
    print("📥 Đang tải xuống...")
else:
    print(f"📁 Dataset đã được lưu tại: {os.path.abspath(output_dir)}")
    print(f"📦 File zip: {os.path.abspath(zip_filename)}")

In [ ]:
# Nén dataset thành file zip
zip_filename = "industrial_product_yolo_dataset.zip"
!cd {output_dir} && zip -r ../{zip_filename} . -q

print(f"✅ Đã tạo file {zip_filename}")

# Download (chỉ trên Colab)
if IN_COLAB:
    from google.colab import files
    files.download(zip_filename)
    print("📥 Đang tải xuống...")
else:
    print(f"📁 Dataset đã được lưu tại: {os.path.abspath(output_dir)}")
    print(f"📦 File zip: {os.path.abspath(zip_filename)}")

In [ ]:
# (Optional) Lưu vào Google Drive
if IN_COLAB:
    save_to_drive = input("Bạn có muốn lưu vào Google Drive? (y/n): ").lower() == 'y'

    if save_to_drive:
        from google.colab import drive
        drive.mount('/content/drive')

        drive_path = "/content/drive/MyDrive/industrial_product_dataset"
        !mkdir -p {drive_path}
        !cp -r {output_dir}/* {drive_path}/
        !cp {zip_filename} {drive_path}/

        print(f"✅ Đã lưu dataset vào Google Drive: {drive_path}")

## 7. (Bonus) Train YOLO Model

Sau khi có dataset, bạn có thể train model YOLO ngay trên Colab.

In [ ]:
# Cài đặt Ultralytics YOLO
!pip install -q ultralytics

from ultralytics import YOLO

# Load pretrained YOLOv8 model
model = YOLO('yolov8n.pt')  # nano model, nhanh nhất
# Có thể dùng: yolov8s.pt (small), yolov8m.pt (medium), yolov8l.pt (large)

print("✅ Đã load YOLOv8 model")

In [ ]:
# Train model
# Bỏ comment dòng dưới để bắt đầu training

# results = model.train(
#     data=os.path.join(output_dir, "data.yaml"),
#     epochs=50,
#     imgsz=640,
#     batch=16,
#     name="product_detector",
#     patience=10,
#     device=0 if torch.cuda.is_available() else 'cpu',
# )

print("💡 Bỏ comment code phía trên để bắt đầu training YOLO model")
print("⏱️ Training có thể mất 30-60 phút tùy thuộc vào GPU và số lượng data")

---

## 📋 Tóm tắt

Notebook này đã thực hiện:

1. ✅ **Setup môi trường** - Cài đặt SAM 2 và dependencies trên Colab
2. ✅ **Video Processing** - Extract video thành frames JPEG
3. ✅ **Interactive Annotation** - Giao diện click chọn đối tượng
4. ✅ **SAM 2 Segmentation** - Segment và tracking tự động qua video
5. ✅ **YOLO Export** - Xuất dataset theo format YOLO (detection/segmentation)
6. ✅ **Download** - Tải dataset về hoặc lưu vào Google Drive

### 📁 Cấu trúc Dataset Output:
```
yolo_product_dataset/
├── images/
│   ├── train/
│   │   ├── 00000.jpg
│   │   └── ...
│   └── val/
├── labels/
│   ├── train/
│   │   ├── 00000.txt
│   │   └── ...
│   └── val/
└── data.yaml
```

### 🚀 Next Steps:
- Train YOLO model với dataset đã tạo
- Fine-tune hyperparameters nếu kết quả chưa tốt
- Augment data nếu cần thêm samples